In [8]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [9]:
from google.colab import files
uploaded = files.upload()

Saving sample.csv to sample (1).csv
Saving train.csv to train (1).csv


In [10]:
train = pd.read_csv('train.csv')
sample = pd.read_csv('sample.csv')
train

,datetime,total
0,01.01.2005 00:00:00,NaN
1,01.01.2005 01:00:00,154139.8084
2,01.01.2005 02:00:00,157818.3593
3,01.01.2005 03:00:00,149310.6991
4,01.01.2005 04:00:00,138282.0380
...,...,...
35059,31.12.2008 19:00:00,249376.3608
35060,31.12.2008 20:00:00,246510.5725
35061,31.12.2008 21:00:00,226469.4133
35062,31.12.2008 22:00:00,199907.3942


In [11]:
sample.info() 
sample['total'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4344 entries, 0 to 4343
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   datetime  4344 non-null   object
 1   total     4344 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 68.0+ KB


array([0])

In [12]:
train['datetime'].unique()

array(['01.01.2005 00:00:00', '01.01.2005 01:00:00',
       '01.01.2005 02:00:00', ..., '31.12.2008 21:00:00',
       '31.12.2008 22:00:00', '31.12.2008 23:00:00'], dtype=object)

In [13]:
train['total']=train['total'].fillna(method='ffill')
train['total']=train['total'].fillna(train['total'].mean())
train

,datetime,total
0,01.01.2005 00:00:00,185144.454732
1,01.01.2005 01:00:00,154139.808400
2,01.01.2005 02:00:00,157818.359300
3,01.01.2005 03:00:00,149310.699100
4,01.01.2005 04:00:00,138282.038000
...,...,...
35059,31.12.2008 19:00:00,249376.360800
35060,31.12.2008 20:00:00,246510.572500
35061,31.12.2008 21:00:00,226469.413300
35062,31.12.2008 22:00:00,199907.394200


In [15]:
sample['day'] = 0
sample['month'] = 0
sample['year'] = 0
sample['time'] = 0

for index, row in train.iterrows():
  train.loc[index, 'day']= str(re.findall(r'^..', train.loc[index, 'datetime']))
  train.loc[index, 'month']=str(re.findall(r'[.][0-1][1-9][.]', train.loc[index, 'datetime']))
  train.loc[index, 'year']=str(re.findall(r'.... ', train.loc[index, 'datetime']))
  train.loc[index, 'time']=str(re.findall(r' ........', train.loc[index, 'datetime']))
train

,datetime,total,day,month,year,time
0,01.01.2005 00:00:00,185144.454732,['01'],['.01.'],['2005 '],[' 00:00:00']
1,01.01.2005 01:00:00,154139.808400,['01'],['.01.'],['2005 '],[' 01:00:00']
2,01.01.2005 02:00:00,157818.359300,['01'],['.01.'],['2005 '],[' 02:00:00']
3,01.01.2005 03:00:00,149310.699100,['01'],['.01.'],['2005 '],[' 03:00:00']
4,01.01.2005 04:00:00,138282.038000,['01'],['.01.'],['2005 '],[' 04:00:00']
...,...,...,...,...,...,...
35059,31.12.2008 19:00:00,249376.360800,['31'],['.12.'],['2008 '],[' 19:00:00']
35060,31.12.2008 20:00:00,246510.572500,['31'],['.12.'],['2008 '],[' 20:00:00']
35061,31.12.2008 21:00:00,226469.413300,['31'],['.12.'],['2008 '],[' 21:00:00']
35062,31.12.2008 22:00:00,199907.394200,['31'],['.12.'],['2008 '],[' 22:00:00']


In [22]:
le_1 = LabelEncoder()
label = le_1.fit_transform(train['day'])

train.drop("day", axis=1, inplace=True)
train["day"] = label

le_2 = LabelEncoder()
label = le_2.fit_transform(train['month'])

train.drop("month", axis=1, inplace=True)
train["month"] = label

le_3 = LabelEncoder()
label = le_3.fit_transform(train['year'])

train.drop("year", axis=1, inplace=True)
train["year"] = label

le_4 = LabelEncoder()
label = le_4.fit_transform(train['time'])

train.drop("time", axis=1, inplace=True)
train["time"] = label

In [24]:

train['total']=pd.to_numeric(train['total'])
train['total']=round(train['total'])
train

,total,day,month,year,time
0,185144.0,0,0,0,0
1,154140.0,0,0,0,1
2,157818.0,0,0,0,2
3,149311.0,0,0,0,3
4,138282.0,0,0,0,4
...,...,...,...,...,...
35059,249376.0,30,10,3,19
35060,246511.0,30,10,3,20
35061,226469.0,30,10,3,21
35062,199907.0,30,10,3,22


In [25]:
X = train.drop(['total'], axis = 1)
y = train['total']

scaler = StandardScaler()
X_st = scaler.fit_transform(X)

X_st

array([[-1.67380491, -1.60078063, -1.34219223, -1.66132477],
       [-1.67380491, -1.60078063, -1.34219223, -1.51686175],
       [-1.67380491, -1.60078063, -1.34219223, -1.37239873],
       ...,
       [ 1.73525035,  1.29620995,  1.34035613,  1.37239873],
       [ 1.73525035,  1.29620995,  1.34035613,  1.51686175],
       [ 1.73525035,  1.29620995,  1.34035613,  1.66132477]])

In [26]:
X = train[['day', 'month', 'year', 'time']].values

y = train['total'].values

features = pd.Series(['day', 'month', 'year', 'time'])
features

0      day
1    month
2     year
3     time
dtype: object

In [27]:
from sklearn.linear_model import LinearRegression
slr = LinearRegression()
slr.fit(X, y)

coeff_df = pd.DataFrame(slr.coef_, columns=['Coefficient'])  
coeff_df['features'] = features

coeff_df

,Coefficient,features
0,84.836221,day
1,-758.721302,month
2,-8055.230927,year
3,4433.629308,time


In [28]:
X = train[['time']].values
y = train['total'].values
slr = LinearRegression()
slr.fit(X, y)
y_pred = slr.predict(X)

In [30]:
for index, row in sample.iterrows():
  sample.loc[index, 'day']= str(re.findall(r'^..', sample.loc[index, 'datetime']))
  sample.loc[index, 'month']=str(re.findall(r'[.][0-1][1-9][.]', sample.loc[index, 'datetime']))
  sample.loc[index, 'year']=str(re.findall(r'.... ', sample.loc[index, 'datetime']))
  sample.loc[index, 'time']=str(re.findall(r' ........', sample.loc[index, 'datetime']))
sample

,datetime,total,day,month,year,time
0,01.01.2009 00:00:00,0,['01'],['.01.'],['2009 '],[' 00:00:00']
1,01.01.2009 01:00:00,0,['01'],['.01.'],['2009 '],[' 01:00:00']
2,01.01.2009 02:00:00,0,['01'],['.01.'],['2009 '],[' 02:00:00']
3,01.01.2009 03:00:00,0,['01'],['.01.'],['2009 '],[' 03:00:00']
4,01.01.2009 04:00:00,0,['01'],['.01.'],['2009 '],[' 04:00:00']
...,...,...,...,...,...,...
4339,30.06.2009 19:00:00,0,['30'],['.06.'],['2009 '],[' 19:00:00']
4340,30.06.2009 20:00:00,0,['30'],['.06.'],['2009 '],[' 20:00:00']
4341,30.06.2009 21:00:00,0,['30'],['.06.'],['2009 '],[' 21:00:00']
4342,30.06.2009 22:00:00,0,['30'],['.06.'],['2009 '],[' 22:00:00']


In [31]:
le_1 = LabelEncoder()
label = le_1.fit_transform(sample['day'])
sample.drop("day", axis=1, inplace=True)
sample["day"] = label

le_2 = LabelEncoder()
label = le_2.fit_transform(sample['month'])
sample.drop("month", axis=1, inplace=True)
sample["month"] = label

le_3 = LabelEncoder()
label = le_3.fit_transform(sample['year'])
sample.drop("year", axis=1, inplace=True)
sample["year"] = label

le_4 = LabelEncoder()
label = le_4.fit_transform(sample['time'])
sample.drop("time", axis=1, inplace=True)
sample["time"] = label

In [32]:
X = sample.drop(['total','datetime'], axis = 1)
y = train['total']

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_st = scaler.fit_transform(X)

X_st



array([[-1.67162811, -1.46686867,  0.        , -1.66132477],
       [-1.67162811, -1.46686867,  0.        , -1.51686175],
       [-1.67162811, -1.46686867,  0.        , -1.37239873],
       ...,
       [ 1.64822659,  1.4571757 ,  0.        ,  1.37239873],
       [ 1.64822659,  1.4571757 ,  0.        ,  1.51686175],
       [ 1.64822659,  1.4571757 ,  0.        ,  1.66132477]])

In [53]:
X = sample['time'].values
y = train['total'].values

y_pred = slr.predict(X)
sample['total']=y_pred

X = sample[['day']].values
y_pred = slr.predict(X)
sample['total']=y_pred

X = sample[['month']].values
y_pred = slr.predict(X)
sample['total']=y_pred

features = pd.Series(['day', 'month', 'year', 'time'])
features


KeyError: ignored

In [49]:
y_pred = slr.predict(X)
sample['total']=y_pred
sample=sample.drop(['day', 'month','year','time'],axis=1)
sample

KeyError: ignored

In [50]:
sample.to_csv('Result_0.csv',index=False)
files.download("Result_0.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>